In [252]:
import json
from datasets import load_dataset
from tree_sitter import Language, Parser
import numpy as np

In [253]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-python'
  ]
)
PY_LANGUAGE = Language('build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)

In [254]:
def custom_read_callable(src_lines):
    def read_callable(byte_offset, point):
        row, column = point
        if row >= len(src_lines) or column >= len(src_lines[row]):
            return None
        return src_lines[row][column:].encode('utf8')
    return read_callable

In [255]:
def labels(src_lines, node, line_num=0):
    resp = []
    for child in node.children:
        if child.type == 'comment':
            continue
        elif child.type == 'block':
            resp += labels(src_lines, child, child.start_point[0])
        elif child.start_point[0] == child.end_point[0] and child.start_point[0] >= line_num:
            resp.append((child.type, src_lines[child.start_point[0]][:-1].replace('    ', '[tab]')))
            line_num = child.start_point[0]
        elif child.start_point[0] >= line_num:
            resp += labels(src_lines, child, child.start_point[0])
            line_num = child.start_point[0]
        line_num += 1
    return resp

In [256]:
def label_code(solution):
    src_lines =  [e+'\n' for e in solution.split('\n') if e]
    root_node = parser.parse(custom_read_callable(src_lines)).root_node
    return labels(src_lines, root_node)

In [257]:
dataset = load_dataset("codeparrot/apps", split="train")

No config specified, defaulting to: apps/all
Found cached dataset apps (/Users/swapnilsharma/.cache/huggingface/datasets/codeparrot___apps/all/0.0.0/04ac807715d07d6e5cc580f59cdc8213cd7dc4529d0bb819cca72c9f8e8c1aa5)


In [258]:
all_data = load_dataset("codeparrot/apps")
# updated_dataset = all_data.map(lambda example: {"labelled_solutions": [label_code(solution) for solution in json.loads(example["solutions"])]}, remove_columns=["problem_id", "input_output", "difficulty", "url", "starter_code"])
# updated_dataset = all_data.map(lambda example: {"labelled_solutions": [label_code(json.loads(example["solutions"])[0])]}, remove_columns=["problem_id", "input_output", "difficulty", "url", "starter_code"])
all_data = all_data.filter(lambda example: example["solutions"] != "")
# all_data = all_data.filter(lambda example: len(label_code(json.loads(example["solutions"])[0])) > 0)
# updated_dataset = all_data.map(lambda example: {"labelled_solutions": [label_code(json.loads(example["solutions"]))]}, remove_columns=["problem_id", "input_output", "difficulty", "url", "starter_code"])
updated_dataset = all_data.map(lambda example: {"labelled_solutions": [np.asarray(label_code(solution)) for solution in json.loads(example["solutions"]) if len(label_code(solution)) > 0]}, remove_columns=["problem_id", "input_output", "difficulty", "url", "starter_code"])


No config specified, defaulting to: apps/all
Found cached dataset apps (/Users/swapnilsharma/.cache/huggingface/datasets/codeparrot___apps/all/0.0.0/04ac807715d07d6e5cc580f59cdc8213cd7dc4529d0bb819cca72c9f8e8c1aa5)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/swapnilsharma/.cache/huggingface/datasets/codeparrot___apps/all/0.0.0/04ac807715d07d6e5cc580f59cdc8213cd7dc4529d0bb819cca72c9f8e8c1aa5/cache-53cf1e86c6ee9cc7.arrow
Loading cached processed dataset at /Users/swapnilsharma/.cache/huggingface/datasets/codeparrot___apps/all/0.0.0/04ac807715d07d6e5cc580f59cdc8213cd7dc4529d0bb819cca72c9f8e8c1aa5/cache-92de84b270017841.arrow


  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/3765 [00:00<?, ?ex/s]

In [259]:
updated_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'solutions', 'labelled_solutions'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['question', 'solutions', 'labelled_solutions'],
        num_rows: 3765
    })
})

In [261]:
updated_dataset.save_to_disk('./code_parrot')

In [244]:
iteration = iter(updated_dataset['train'])

In [245]:
sample = next(iteration)

In [249]:
sample['labelled_solutions']

AttributeError: 'list' object has no attribute 'size'

In [173]:
sample["solutions"] == ''

True

In [206]:
# sample = next(dataset)
sample = dataset[827]

In [207]:
sample.keys()

dict_keys(['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'])

In [208]:
# print(sample['question'])

In [187]:
len(json.loads(sample["solutions"]))

3

In [188]:
# print(json.loads(sample["solutions"])[0])

In [178]:
for sample in dataset:
    label_code(json.loads(sample["solutions"])[0])

In [209]:
print(json.loads(sample["solutions"])[0])

try:
    t=int(input())
    for i in range(t):
        n,k=map(int,input().split())
        s=input()
        l=[-1]*len(s)
        numb=s.count('b')
        x=numb
        for j in range(len(s)):
            if(s[j]=='a'):
                l[j]=numb
            if(s[j]=='b'):
                numb=numb-1
        #print(l)
        count1=0
        for j in range(len(l)):
            if(l[j]>0):
                count1=count1+(k*(2*l[j]+(k-1)*x))//2
            elif(l[j]==0):
                count1=count1+(k*(2*0+(k-1)*x))//2
        print(count1)
except:
    pass



In [210]:
solution = json.loads(sample["solutions"])[0]

In [211]:
src_lines =  [e+'\n' for e in solution.split('\n') if e]
root_node = parser.parse(custom_read_callable(src_lines)).root_node
# return np.asarray(labels(src_lines, root_node))

In [212]:
src_lines

['try:\r\n',
 '    t=int(input())\r\n',
 '    for i in range(t):\r\n',
 '        n,k=map(int,input().split())\r\n',
 '        s=input()\r\n',
 '        l=[-1]*len(s)\r\n',
 "        numb=s.count('b')\r\n",
 '        x=numb\r\n',
 '        for j in range(len(s)):\r\n',
 "            if(s[j]=='a'):\r\n",
 '                l[j]=numb\r\n',
 "            if(s[j]=='b'):\r\n",
 '                numb=numb-1\r\n',
 '        #print(l)\r\n',
 '        count1=0\r\n',
 '        for j in range(len(l)):\r\n',
 '            if(l[j]>0):\r\n',
 '                count1=count1+(k*(2*l[j]+(k-1)*x))//2\r\n',
 '            elif(l[j]==0):\r\n',
 '                count1=count1+(k*(2*0+(k-1)*x))//2\r\n',
 '        print(count1)\r\n',
 'except:\r\n',
 '    pass\r\n']

In [213]:
root_node.sexp()

'(module (try_statement body: (block (expression_statement (assignment left: (identifier) right: (call function: (identifier) arguments: (argument_list (call function: (identifier) arguments: (argument_list)))))) (for_statement left: (identifier) right: (call function: (identifier) arguments: (argument_list (identifier))) body: (block (expression_statement (assignment left: (pattern_list (identifier) (identifier)) right: (call function: (identifier) arguments: (argument_list (identifier) (call function: (attribute object: (call function: (identifier) arguments: (argument_list)) attribute: (identifier)) arguments: (argument_list)))))) (expression_statement (assignment left: (identifier) right: (call function: (identifier) arguments: (argument_list)))) (expression_statement (assignment left: (identifier) right: (binary_operator left: (list (unary_operator argument: (integer))) right: (call function: (identifier) arguments: (argument_list (identifier)))))) (expression_statement (assignmen

In [226]:
label_code(json.loads(sample["solutions"])[0]).size

40

## Load wikihow data

In [17]:
!ls ./wikihow

__init__.py       wiki_how_data.pkl


In [18]:
import pickle

# open a file, where you stored the pickled data
file = open('./wikihow/wiki_how_data.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

print('Showing the pickled data:')

Showing the pickled data:


In [22]:
data[0].keys()

dict_keys(['url', 'title', 'description', 'steps'])

In [23]:
data[0]['description']

'Nearly one third of all driving accidents in the United States involve alcohol-impaired drivers. Driving with a blood alcohol level of .08 or more is a crime.  You may also be unable to adequately operate a motor vehicle with a much lower level, which puts the driver and others at risk. Drunk driving is avoidable if you exercise precautions and self-awareness to stay safe.'

In [24]:
data[0]['title']

'How to Avoid Drinking and Driving'

In [35]:
list(data[0]['steps'].values())[1]

{'section': 'Designating a Driver',
 'step_num': '2',
 'summary': 'Assign a designated driver.',
 'text': 'Make sure this person is responsible and keep them in your sight while you are drinking.  Have their contact information handy in case you can’t find them when you are ready to leave.  Choose a friend who doesn’t drink alcohol.  You likely have someone in your friend group who doesn’t drink.  This person is the most likely to remain sober. Decide on one person who will remain sober.  You can take turns within your friend group, alternating who will be the designated driver on each occasion.  Be sure that the designated driver actually remains sober.  The person who has drank the least is still not sober.'}

In [36]:
list(data[0]['steps'].values())[1]['summary'][:-1]

'Assign a designated driver'